In [36]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [37]:
creditCustomers = pd.read_csv("../data/project/ds.csv")
#creditCustomers

creditcustomers_small = creditCustomers.iloc[:,1:]
creditcustomers_small = creditcustomers_small.rename(columns={'FICO V2 Score':'F02_SCORE',  'FICO V3 Score':'F03_SCORE', 'FICO V08 Score':'F08_SCORE'})
creditcustomers_small = creditcustomers_small.loc[creditcustomers_small['F02_SCORE'] < 1000]

#Removing customers with credit score > 800 and declined
creditcustomers_small = creditcustomers_small.loc[(creditcustomers_small['F02_SCORE'] < 800) 
                | ((creditcustomers_small['F02_SCORE'] >= 800) & (creditcustomers_small['Status'] == 'APPROVED'))]
creditcustomers_small = creditcustomers_small.loc[(creditcustomers_small['F03_SCORE'] < 800) 
                | ((creditcustomers_small['F03_SCORE'] >= 800) & (creditcustomers_small['Status'] == 'APPROVED'))]
creditcustomers_small = creditcustomers_small.loc[(creditcustomers_small['F08_SCORE'] < 800) 
                | ((creditcustomers_small['F08_SCORE'] >= 800) & (creditcustomers_small['Status'] == 'APPROVED'))]

#No need to run the below as all that have score > 1000 have the same value for all columns
#creditcustomers_small = creditcustomers_small.loc[creditcustomers_small['F03_SCORE'] < 1000]
#creditcustomers_small = creditcustomers_small.loc[creditcustomers_small['F08_SCORE'] < 1000]

#creditcustomers_small

In [38]:
#Replacing status APPROVED to 1
creditcustomers_small.replace(to_replace={'Status':"APPROVED"}, value=1, inplace=True)
#Replacing status DECLINED to 0
creditcustomers_small.replace(to_replace={'Status':"DECLINED"}, value=0, inplace=True)

#creditcustomers_small

In [39]:
fv = creditcustomers_small[['F02_SCORE', 'F03_SCORE', 'F08_SCORE']].values
labels = creditcustomers_small['Status'].values

In [87]:
 from sklearn.model_selection import train_test_split
## Split into training and test
## TODO: create training and test with an 80/20 split
X_train, X_test, Y_train, Y_test = train_test_split(creditcustomers_small[['F02_SCORE', 'F03_SCORE', 'F08_SCORE']]
                                                    , labels, test_size=0.3)


print ("training set count ", len(X_train))
print ("testing set count ", len(Y_test))

training set count  395
testing set count  170


In [88]:
from sklearn import svm
import time

clf = svm.SVC(kernel='linear')

t1 = time.perf_counter()
clf.fit(X_train, Y_train)
t2 = time.perf_counter()


print("traind on {:,} records in {:,.2f} ms".\
      format(len(X_train), (t2-t1)*1000))

# Print the coefficients and intercept for linearsSVC
print ("inputs : F02_SCORE, F03_SCORE, F08_SCORE")

traind on 395 records in 1,532.46 ms
inputs : F02_SCORE, F03_SCORE, F08_SCORE


In [89]:
predictions = clf.predict(X_test)

In [93]:
from sklearn.metrics import accuracy_score
print ("Training set count: ", len(X_train), "/ 70%")
print ("Testing set count: ", len(Y_test), "/ 30%")
print("Accuracy = " + str(accuracy_score(Y_test, predictions)))


Training set count:  395 / 70%
Testing set count:  170 / 30%
Accuracy = 0.7647058823529411


In [91]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test, predictions)

0.76